In [1]:
import pandas as pd
import re
import numpy as np
import random

In [2]:
ori_train_df = pd.read_csv('../../patent_baseline/train.csv')
train_df = ori_train_df.copy()

# 모양 처리
form_entity : 모양과 관련된 기호를 추가했습니다
form_sub_entity : 형상, 형 etc...

regex_form() 함수를 사용하면 모양관련 처리가 됩니다.



## 사용하실 때 복사해야 할 부분 
form_entity , form_sub_entity, regex_form() 있는 페이지

ex) train_df['abstract'] = train_df['abstract'].apply(lambda x: regex_form(x))

In [16]:
form_entity = """
∧
⊥
┘
┗
□
┳
∃
╋
┛
†
ㅜ
┌
ㅗ
└
L
┐
┓
┏
┣
∠
￢
┍
┚
Π
╉
Ⅰ
|
ㅣ
Ｉ
｜
ㅒ
ㅡ
━
ㅓ
─
∞
／
＼
V
+
⊂
<
(
〈
〔
""".split('\n')[1:-1]

form_sub_entity = """
형태
모양
형상
형사
형
자
""".split('\n')[1:-1]

In [17]:
def find_special_form(text, form_entity):
    find = any(special in text for special in form_entity)
    return [find, text]  # Return a list with the result and text

form_abstract = train_df['abstract'].apply(lambda x: find_special_form(x, form_entity))

# Convert the result to a DataFrame with columns 'find' and 'text'
form_abstract = pd.DataFrame(form_abstract.tolist(), columns=['find', 'abstract'])


In [18]:
form_abstract['abstract'][1196]

'본고안은 포장용상자에 관한 것이다.더욱 상세하게는 포장상자의 옆판에 상부는 절첩되어있고 하부는 벌어져있는 상태로 형성된 ∧ 형의 절곡부를 구성하여 포장상자내의 내용물이 안쪽에서 밀어주는 작용을 하므로서 상자내의 내용물이 쏟아지지 않으며,또한 뚜껑부에 손잡이와 일체로 형성된 체결부위가 체결공에 밑으로부터 위쪽으로 체결되는 체결방법으로인하여 체결공에 체결된 체결부위가 이탈되지 않토록 구성된 것이다.'

In [19]:
form_temp = form_abstract[form_abstract['find'] == True]
idx_form = form_temp.index.tolist()
print(idx_form)

[2, 5, 7, 23, 80, 81, 103, 117, 119, 163, 193, 209, 235, 267, 311, 323, 335, 350, 364, 375, 386, 387, 403, 413, 426, 440, 448, 477, 502, 518, 525, 548, 586, 596, 600, 605, 608, 620, 662, 668, 676, 684, 706, 718, 743, 764, 777, 804, 808, 826, 836, 837, 850, 864, 865, 866, 878, 944, 967, 986, 987, 1053, 1076, 1151, 1159, 1163, 1187, 1196, 1203, 1245, 1250, 1252, 1303, 1341, 1343, 1368, 1388, 1403, 1404, 1434, 1471, 1481, 1503, 1535, 1554, 1560, 1561, 1565, 1582, 1616, 1636, 1646, 1718, 1722, 1729, 1739, 1748, 1770, 1790, 1796, 1801, 1825, 1842, 1854, 1870, 1955, 1983, 1988, 1994, 2015, 2018, 2022, 2102, 2119, 2128, 2130, 2134, 2137, 2155, 2198, 2226, 2243, 2263, 2292, 2300, 2338, 2344, 2363, 2417, 2425, 2427, 2429, 2432, 2470, 2477, 2478, 2479, 2524, 2540, 2550, 2554, 2566, 2569, 2596, 2633, 2641, 2653, 2668, 2692, 2708, 2734, 2743, 2787, 2801, 2818, 2833, 2908, 2918, 2928, 2937, 2965, 2988, 2991, 3010, 3020, 3021, 3054, 3085, 3104, 3108, 3117, 3120, 3135, 3136, 3144, 3162, 3190, 3192, 3

In [20]:
# # 정규표현식 패턴 생성
pattern_form = '|'.join(re.escape(symbol) for symbol in form_entity)
pattern_sub = '|'.join(text for text in form_sub_entity)

# 패턴에 매치되는 부분 찾기
def find_form(pattern,text):
    arr = []
    matches = re.finditer(pattern, text)

    #결과 출력
    for match in matches:
        arr.append([match.start(),match.end()])
        # print(f"기호 '{match.group()}'이(가) 발견된 위치: {match.start()}-{match.end()}")
    return arr

def regex_form(text):
    new_Text = "모양"
    modified_text = text
    del_len = 0
    try:
        # 기호 , 단어 위치 찾기
        matches_form = find_form(pattern_form,text)
        matches_sub = find_form(pattern_sub,text)
        # 단어가 기호의 마지막 index + 5 이내에 있으면 출력
        for match_form in matches_form:
            for match_sub in matches_sub:
                if match_form[0] <= match_sub[1] <= match_form[1]+5:
                    start = match_form[0] - del_len
                    end = match_sub[1] - del_len
                    print(f"범위 {match_form[0]}~{match_sub[1]}")
                    modified_text = modified_text[:start] + new_Text + modified_text[end:]
                    del_len = del_len + (match_sub[1] - match_form[0]+1) - len(new_Text) - 1
                    break
    except:
        modified_text = text        
    return modified_text

# # # test
# print(regex_form(form_temp['abstract'][1955]))
# print(form_temp['abstract'][1955])
print(regex_form("본 고안은 다목적 증기 온수 발생장치에서 증기 또는 온수가 배출되는 호스 단부에 자바라 형태의 주름부를 형성하고, 주름부의 최선단에 구형(求刑)이나 반구형(半球形) 또는 이와 유사한 외면 형상의 노즐을 형성하거나 설치하여 ┛ 형상, ┳ 형상, ╋ 형사, ⊂ 형상, 〈 형상, 〔 형상, ∃ 형상 등의 직각관이나 꺽임이 심한 곡선관, 관체의 단턱부, 또는 엘보(elbow)부분에도 쉽게 삽입시켜 해빙(解氷)할 수 있도록 한 것이다."))

범위 76~76
범위 86~86
범위 124~128
범위 130~134
범위 136~140
범위 142~146
범위 148~152
범위 154~158
범위 160~164
본 고안은 다목적 증기 온수 발생장치에서 증기 또는 온수가 배출되는 호스 단부에 자바라 형태의 주름부를 형성하고, 주름부의 최선단에 구형모양(求刑)이나 반구형모양(半球形) 또는 이와 유사한 외면 형상의 노즐을 형성하거나 설치하여 모양, 모양, 모양, 모양, 모양, 모양, 모양 등의 직각관이나 꺽임이 심한 곡선관, 관체의 단턱부, 또는 엘보(elbow)부분에도 쉽게 삽입시켜 해빙(解氷)할 수 있도록 한 것이다.


In [21]:
form_temp['extrected'] = form_temp['abstract'].apply(lambda x: regex_form(x))

범위 120~122
범위 177~179
범위 244~246
범위 361~363
범위 99~102
범위 134~140
범위 67~70
범위 105~108
범위 219~222
범위 235~238
범위 135~137
범위 121~123
범위 15~20
범위 69~75
범위 189~192
범위 251~254
범위 267~269
범위 366~369
범위 429~432
범위 445~447
범위 283~285
범위 29~31
범위 467~469
범위 122~125
범위 171~177
범위 173~177
범위 541~546
범위 543~546
범위 500~504
범위 36~40
범위 154~156
범위 324~326
범위 144~146
범위 161~164
범위 69~72
범위 263~266
범위 284~287
범위 288~290
범위 446~451
범위 74~76
범위 82~84
범위 235~237
범위 69~71
범위 299~301
범위 411~413
범위 181~185
범위 190~192
범위 272~274
범위 284~286
범위 367~369
범위 506~508
범위 200~202
범위 257~259
범위 375~377
범위 504~506
범위 565~567
범위 468~470
범위 96~98
범위 211~213
범위 404~407
범위 529~532
범위 273~275
범위 116~120
범위 18~23
범위 169~171
범위 247~249
범위 171~177
범위 587~593
범위 206~208
범위 258~260
범위 561~563
범위 604~606
범위 316~321
범위 472~474
범위 205~207
범위 248~250
범위 357~359
범위 183~185
범위 177~181
범위 135~138
범위 221~224
범위 297~300
범위 75~79
범위 60~64
범위 453~455
범위 490~492
범위 18~20
범위 269~271
범위 303~305
범위 103~109
범위 153~158
범위 221~223
범위 353~355
범위 185

C:\Users\hyundong\AppData\Local\Temp\ipykernel_21668\3180563456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form_temp['extrected'] = form_temp['abstract'].apply(lambda x: regex_form(x))


In [22]:
form_temp['extrected'][1196]

'본고안은 포장용상자에 관한 것이다.더욱 상세하게는 포장상자의 옆판에 상부는 절첩되어있고 하부는 벌어져있는 상태로 형성된 모양의 절곡부를 구성하여 포장상자내의 내용물이 안쪽에서 밀어주는 작용을 하므로서 상자내의 내용물이 쏟아지지 않으며,또한 뚜껑부에 손잡이와 일체로 형성된 체결부위가 체결공에 밑으로부터 위쪽으로 체결되는 체결방법으로인하여 체결공에 체결된 체결부위가 이탈되지 않토록 구성된 것이다.'

In [ ]:
form_check = form_temp.drop('abstract',axis=1)
form_check.to_csv('./text.csv')